In [1]:
%display latex

In [2]:
eps = var('eps', latex_name = '\\varepsilon')
t = var('t')
r = var('r')
th = var('th', latex_name = '\\theta')
ph = var('ph', latex_name = '\\varphi')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name='\\omega')(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne', latex_name='N_e')(r,eps)

In [3]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for kk in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for kk in range(0,4):
                chris[i][j][kk] = 0.5*sum(geuu[i,dd]*(diff(gedd[dd,j],coords[kk])
                                                   + diff(gedd[dd,kk],coords[j])
                                                   - diff(gedd[j,kk],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for kk in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for kk in range(0,4):
                for l in range(0,4):
                    riem[i][j][kk][l] = (diff(chris[i][j][l],coords[kk])
                                      - diff(chris[i][j][kk],coords[l])
                                      + sum(chris[i][kk][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][kk][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for kk in range(0,4):
            ricci[i,kk] = sum(riem[dd][i][dd][kk] for dd in range(0,4))

    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    Ge = matrix(SR, 4, 4)
    Ge = ricci - 0.5*gedd*s_curv
    
    dGe = matrix(SR, 4, 4)
    ddGe = matrix(SR, 4, 4)
    for i in range(0,4):
        for j in range(0,4):
            dGe[i,j] = diff(Ge[i,j],eps)
            ddGe[i,j] = diff(Ge[i,j],eps,eps)
    
    return {'inverse_metric': geuu,
            'einstein': Ge,
            'd_einstein': dGe,
            'dd_einstein': ddGe}

In [4]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(i,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]
        gedd[j,i] = gedd[i,j]

tensors = cosmotensors(gedd)

geuu = tensors['inverse_metric']
Ge = tensors['einstein']
dGe = tensors['d_einstein']
ddGe = tensors['dd_einstein']

In [5]:
show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                          0                           0                           0 -r^2*omega(r, th)*sin(th)^2]
[                          0                           0                           0                           0]
[                          0                           0                           0                           0]
[-r^2*omega(r, th)*sin(th)^2                           0                           0                           0]

'K2 = ' [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

'ge = ' [(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*eps^2 - e^nu(r)                                                                   0                                                                   0                                     -eps*r^2*omega(r, th)*sin(th)^2]
[                                                                  0                              2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                   0                                                                   0]
[                                                                  0                                                                   0                                          2*eps^2*r^2*k(r, th) + r^2                                                                   0]
[                                    -eps*r^2*omega(r, th)*sin(th)^2                                                                   0                                                                   0                      2*eps^2*r^2*k(r, th)*sin(th)^2 + r^2*sin(th)^2]

In [6]:
#4-velocities of the fluid for any epsilon

ue = [0,0,0,0]
u = [0,0,0,0]

ue[0] = Ne;         ue[1] = 0;   ue[2] = 0;   ue[3] = Ne*Omegae
u[0] = exp(-nu/2);  u[1] =  0;   u[2] = 0;    u[3] =  0

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                             #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                                     #Eq 4.20

In [7]:
show(LatexExpr("N_{\\varepsilon}\lvert_0 = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\lvert_0 = "), Omegaevalue)

N_{\varepsilon}\lvert_0 =  e^(-1/2*nu(r))

\Omega_{\varepsilon}\lvert_0 =  0

In [8]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10',latex_name="u^{(1)0}")(r,th)
u20 = function('u20',latex_name="u^{(2)0}")(r,th)

Omega1 = var('Omega1', latex_name="\\Omega^{(1)}")
Omega2 = var('Omega2', latex_name="\\Omega^{(2)}")

s0 = eps == 0

s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue

s3 = diff(Ne,eps,eps).subs(eps==0) == u20
s4 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

s5 = diff(Ne,eps).subs(eps==0) == u10
s6 = diff(Omegae,eps).subs(eps==0) == Omega1

sustituciones = [s0,s1,s2,s3,s4,s5,s6]

for i in range(0,4):
    u1[i] = diff(ue[i],eps)
    u2[i] = diff(ue[i],eps,eps)

Ue = [0,0,0,0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[j]*gedd[j,i]

geUeue = 0
for i in range(0,4):
    geUeue += Ue[i]*ue[i]

ddgeUeue = diff(geUeue, eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s7 = u20 == u20value

dgeUeue = diff(geUeue, eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s8 = u10 == u10value

sustituciones.extend([s7,s8])
for i in range(0,4):
    for kk in range(0,len(sustituciones)):
        u1[i] = u1[i].subs(sustituciones[kk])
        u2[i] = u2[i].subs(sustituciones[kk])

In [9]:
show(LatexExpr("u^{(1)} = "), u1)                                                                         #Eq 4.21
show(LatexExpr("u^{(2)} = "), u2)                                                                         #Eq 4.22

u^{(1)} =  [0, 0, 0, Omega1*e^(-1/2*nu(r))]

u^{(2)} =  [((Omega1^2*r^2 - 2*Omega1*r^2*omega(r, th) + r^2*omega(r, th)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)),
 0,
 0,
 Omega2*e^(-1/2*nu(r))]

In [10]:
# CHECK
u1_paper = [0,0,0,0]
u1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
u1_paper[3] = e^(-nu/2)*Omega1

u2_paper = [0,0,0,0]
u2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omega1*K1[0,3]+Omega1^2*r^2*sin(th)^2)
u2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omega1+Omega2)  

u1_sum = 0
u2_sum = 0
for i in range(0,4):
    u1_sum += u1[0] - u1_paper[0]
    u2_sum += u2[0] - u2_paper[0]

u1_sum = u1_sum.simplify_full()
u2_sum = u2_sum.simplify_full()

show(u1_sum)
show(u2_sum)

0

0

In [11]:
G = copy(Ge)
for kk in range(0,len(sustituciones)):
    G = G.subs(sustituciones[kk])

In [12]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,th,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,th,eps)

Te = matrix(SR, 4, 4)
dTe = matrix(SR, 4, 4)
ddTe = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        Te[i,j] = (Ee+Pe)*Ue[i]*Ue[j]+Pe*gedd[i,j]
        dTe[i,j] = diff(Te[i,j],eps)
        ddTe[i,j] = diff(Te[i,j],eps,eps)

T = copy(Te)
for kk in range(0,len(sustituciones)):
    T = T.subs(sustituciones[kk])

In [13]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), T)

T_{\varepsilon}\lvert_0 = [(Ee(r, th, 0) + Pe(r, th, 0))*e^nu(r) - Pe(r, th, 0)*e^nu(r)                                                            0                                                            0                                                            0]
[                                                           0                                       Pe(r, th, 0)*e^lamb(r)                                                            0                                                            0]
[                                                           0                                                            0                                             r^2*Pe(r, th, 0)                                                            0]
[                                                           0                                                            0                                                            0                                   r^2*Pe(r, th, 0)*sin(th)^2]

In [14]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones.extend([s9,s10])

T = T.subs(s9).subs(s10)
        
Evalue = solve(G[0,0]==kappa*T[0,0],E)[0].rhs()                                                          #Eq 3.7
Pvalue = solve(G[1,1]==kappa*T[1,1],P)[0].rhs()                                                          #Eq 3.8
Pvalue2 = solve(G[2,2]==kappa*T[2,2],P)[0].rhs()
Pvalue3 = solve(G[3,3]==kappa*T[3,3],P)[0].rhs()
ddnu = solve(Pvalue == Pvalue2, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [15]:
show(LatexExpr("E = "), Evalue)
show(LatexExpr("P_{rr} = "), Pvalue)
show(LatexExpr("\\nu'' ="), ddnu)

s11 = E == Evalue
s12 = P == Pvalue
s13 = diff(nu,r,r) == ddnu
sustituciones.extend([s11,s12,s13])

E =  1/8*(r*e^(-lamb(r))*diff(lamb(r), r) - e^(-lamb(r)) + 1)/(pi*r^2)

P_{rr} =  1/8*(r*e^(-lamb(r))*diff(nu(r), r) + e^(-lamb(r)) - 1)/(pi*r^2)

\nu'' = -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [16]:
Evalue_paper = diff(lamb,r) - 1/r*(1-e^(lamb)) - r*e^(lamb)*8*pi*E == 0
Evalue_paper = solve(Evalue_paper, E)[0].rhs()

Evalue_check = Evalue - Evalue_paper
show(Evalue_check.simplify_full())

Pvalue_paper = diff(nu,r) - 1/r*(-1+e^(lamb)) - r*e^(lamb)*8*pi*P == 0
Pvalue_paper = solve(Pvalue_paper, P)[0].rhs()

Pvalue_check = Pvalue - Pvalue_paper
show(Pvalue_check.simplify_full())

ddnu_paper = 2*r*diff(nu,r,r) + diff(nu,r)*(r*diff(nu,r)-2)-diff(lamb,r)*(2+r*diff(nu,r))+4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, diff(nu,r,r))[0].rhs()

ddnu_check = ddnu - ddnu_paper
show(ddnu_check.simplify_full())

0

0

0

## Primera derivada

In [17]:
E1 = function('E1', latex_name='E^{(1)}')(r,th)
P1 = function('P1', latex_name='P^{(1)}')(r,th)

In [18]:
s14 = diff(Ee,eps).subs(eps==0) == E1
s15 = diff(Pe,eps).subs(eps==0) == P1

sustituciones.extend([s14,s15])

In [19]:
dG = copy(dGe)
dT = copy(dTe)
for kk in range(0,len(sustituciones)):
    dG = dG.subs(sustituciones[kk])
    dT = dT.subs(sustituciones[kk])

In [20]:
dEq11 = dG[1,1] - kappa*dT[1,1] == 0
dEq11 = dEq11.simplify_full()

dEq22 = dG[2,2] - kappa*dT[2,2] == 0
dEq22 = dEq22.simplify_full()

dEq33 = dG[3,3] - kappa*dT[3,3] == 0
dEq33 = dEq33.simplify_full()

show(dEq11)
show(dEq22)
show(dEq33)

P1value = solve(dEq11,P1)[0].rhs()
sustituciones.extend([P1==P1value])

show(LatexExpr("\\text{De aquí sacamos }P_1(r) \\rightarrow P_1(r) = "), P1value)

-8*pi*P1(r, th)*e^lamb(r) == 0

-8*pi*r^2*P1(r, th) == 0

-8*pi*r^2*P1(r, th)*sin(th)^2 == 0

\text{De aquí sacamos }P_1(r) \rightarrow P_1(r) =  0

In [21]:
dEq00 = dG[0,0] - kappa*dT[0,0] == 0
dEq00 = dEq00.simplify_full()
show(dEq00)

for kk in range(0,len(sustituciones)):
    dEq00 = dEq00.subs(sustituciones[kk])

E1value = solve(dEq00,E1)[0].rhs()
sustituciones.extend([E1==E1value])

show(LatexExpr("\\text{De aquí sacamos }E_1(r,\\theta) \\rightarrow E_1(r,\\theta) = "), E1value)

-8*pi*E1(r, th)*e^nu(r) == 0

\text{De aquí sacamos }E_1(r,\theta) \rightarrow E_1(r,\theta) =  0

In [22]:
dEq03 = dG[0,3] - kappa*dT[0,3]

for kk in range(0,len(sustituciones)):
    dEq03 = dEq03.subs(sustituciones[kk])
    
dEq03 = dEq03.simplify_full()

In [23]:
# CHECK EQUATION 48
jj = e^(-(lamb+nu)/2)
dEq03_paper = diff(r^4*jj*diff(omega,r),r)+\
              (r^2*jj*e^(lamb)/sin(th)^3)*diff(sin(th)^3*diff(omega,th),th)+\
              4*r^3*diff(jj,r)*(omega-Omega1)

dEq03_paper = dEq03_paper*sin(th)^2/(2*r^2*e^(lamb/2-nu/2))

dEq03_check = dEq03_paper - dEq03

dEq03_check.simplify_full()

0

## Segunda derivada

In [24]:
E2 = function('E2', latex_name='E^{(2)}')(r,th)
P2 = function('P2', latex_name='P^{(2)}')(r,th)

In [25]:
s16 = diff(Ee,eps,eps).subs(eps==0) == E2
s17 = diff(Pe,eps,eps).subs(eps==0) == P2

sustituciones.extend([s16,s17])

In [26]:
for i in range(0,len(sustituciones)):
    show(sustituciones[i])
    
sustit = copy(sustituciones)

eps == 0

Ne(r, 0) == e^(-1/2*nu(r))

Omegaevalue(0) == 0

D[1, 1](Ne)(r, 0) == u20(r, th)

D[0, 0](Omegaevalue)(0) == Omega2

...

D[2](Pe)(r, th, 0) == P1(r, th)

P1(r, th) == 0

E1(r, th) == 0

D[2, 2](Ee)(r, th, 0) == E2(r, th)

D[2, 2](Pe)(r, th, 0) == P2(r, th)

In [27]:
ddG = copy(ddGe)
ddT = copy(ddTe)
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[kk])
    ddT = ddT.subs(sustituciones[kk])

In [28]:
E2*diff(P,r) - P2*diff(E,r) == 0

-P2(r, th)*diff(E(r, th), r) + E2(r, th)*diff(P(r, th), r) == 0

### Descomposición en polinomios de Legendre

In [29]:
h0 = function('h0', latex_name='h_0')(r)
k0 = function('k0', latex_name='k_0')(r)
m0 = function('m0', latex_name='m_0')(r)

h1 = function('h1', latex_name='h_1')(r)
k1 = function('k1', latex_name='k_1')(r)
m1 = function('m1', latex_name='m_1')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
m2 = function('m2', latex_name='m_2')(r)

E02 = function('E02', latex_name='E_{0}^{(2)}')(r)
P02 = function('P02', latex_name='P_{0}^{(2)}')(r)

E12 = function('E12', latex_name='E_{1}^{(2)}')(r)
P12 = function('P12', latex_name='P_{1}^{(2)}')(r)

E22 = function('E22', latex_name='E_{2}^{(2)}')(r)
P22 = function('P22', latex_name='P_{2}^{(2)}')(r)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

In [30]:
sustituciones = [m==m0+m1*LP1+m2*LP2,
                 diff(m,r)==diff(m0,r)+diff(m1,r)*LP1+diff(m2,r)*LP2,
                 diff(m,r,r)==diff(m0,r,r)+diff(m1,r,r)*LP1+diff(m2,r,r)*LP2,
                 diff(m,th)==-sin(th)*m1-3*LP1*sin(th)*m2,
                 diff(m,th,th)==-m1*LP1+m2-4*m2*LP2,
                 h==h0+h1*LP1+h2*LP2,
                 diff(h,r)==diff(h0,r)+diff(h1,r)*LP1+diff(h2,r)*LP2,
                 diff(h,r,r)==diff(h0,r,r)+diff(h1,r,r)*LP1+diff(h2,r,r)*LP2,
                 diff(h,th)==-sin(th)*h1-3*LP1*sin(th)*h2,
                 diff(h,th,th)==-h1*LP1+h2-4*h2*LP2,
                 k==k1*LP1+k2*LP2,
                 diff(k,r)==diff(k1,r)*LP1+diff(k2,r)*LP2,
                 diff(k,r,r)==diff(k1,r,r)*LP1+diff(k2,r,r)*LP2,
                 diff(k,th)==-sin(th)*k1-3*LP1*sin(th)*k2,
                 diff(k,th,th)==-k1*LP1+k2-4*k2*LP2,
                 E2==E02+E12*LP1+E22*LP2,
                 diff(E2,r)==diff(E02,r)+diff(E12,r)*LP1+diff(E22,r)*LP2,
                 diff(E2,r,r)==diff(E02,r,r)+diff(E12,r,r)*LP1+diff(E22,r,r)*LP2,
                 diff(E2,th)==-sin(th)*E12-3*LP1*sin(th)*E22,
                 diff(E2,th,th)==-E12*LP1+E22-4*E22*LP2,
                 P2==P02+P12*LP1+P22*LP2,
                 diff(P2,r)==diff(P02,r)+diff(P12,r)*LP1+diff(P22,r)*LP2,
                 diff(P2,r,r)==diff(P02,r,r)+diff(P12,r,r)*LP1+diff(P22,r,r)*LP2,
                 diff(P2,th)==-sin(th)*P12-3*LP1*sin(th)*P22,
                 diff(P2,th,th)==-P12*LP1+P22-4*P22*LP2]

sus = copy(sustituciones)
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[kk])
    ddT = ddT.subs(sustituciones[kk])

In [31]:
s1 = diff(LP1, th, th) == -LP1
s2 = diff(LP2, th) == -3*LP1*sin(th)
s3 = diff(LP2, th, th) == 1-4*LP2
s4 = sin(th)^2 == 2/3*(1-LP2)
s5 = cos(th) == LP1
s6 = LP1^2 == cos(th)^2
s7 = cos(th)^2 == (2*LP2+1)/3

sustituciones = [s1,s2,s3,s4,s5,s6,s7]

In [32]:
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[kk])
    ddT = ddT.subs(sustituciones[kk])

# l=0

In [33]:
ddG = ddG.subs(LP1==0).subs(LP2==0)
ddT = ddT.subs(LP1==0).subs(LP2==0)

In [34]:
ddG[0,0] = ddG[0,0].full_simplify()
ddT[0,0] = ddT[0,0].full_simplify()

In [35]:
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[kk])
    ddT = ddT.subs(sustituciones[kk])
    
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sus[kk])
    ddT = ddT.subs(sus[kk])
    
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[kk])
    ddT = ddT.subs(sustituciones[kk])
    
for kk in range(0,len(sustituciones)):
    ddG = ddG.subs(sus[kk])
    ddT = ddT.subs(sus[kk])

In [36]:
ddEq00 = ddG[0,0] - kappa*ddT[0,0] == 0
E02value = solve(ddEq00, E02)[0].rhs()

E02value_paper = ((4/r^2)*diff(r*e^(-lamb)*m0,r)\
                  +8/3*r*jj*diff(jj,r)*(omega-Omega1)^2\
                  -1/3*jj^2*r^2*diff(omega,r)^2)/(8*pi)

for kk in range(0,len(sustit)):
    E02value_paper = E02value_paper.subs(sustit[kk])

E02value_check = (E02value - E02value_paper)
E02value_check = E02value_check.simplify_full()
E02value_check.simplify_full()

-1/162129586585337856*(4503599627370496*r^4*diff(omega(r, th), r)^2 + 18014398509481984*r^4*omega(r, th)*diff(omega(r, th), r, r) + 10133099161583616*(r^3*omega(r, th)^2*diff(lamb(r), r) + 3*r^3*omega(r, th)^2*diff(nu(r), r) - 4*r^3*omega(r, th)*diff(omega(r, th), r) - 8*r^2*omega(r, th)^2)*LP1(th)^4 + 4503599627370496*r^2*e^lamb(r)*diff(omega(r, th), th)^2 + 18014398509481984*r^2*e^lamb(r)*omega(r, th)*diff(omega(r, th), th, th) + 6*r^2*e^nu(r)*diff(h0(r), r, r) - 54043195528445952*(e^lamb(r) - 1)*e^nu(r)*h0(r) + 12*(e^lamb(r) - 4503599627370496)*e^nu(r)*m0(r) + 4503599627370496*(11*r^2*e^lamb(r) - 2*r^2)*omega(r, th)^2 - 6755399441055744*(r^4*diff(omega(r, th), r)^2 - 12*(e^lamb(r) - 1)*e^nu(r)*h0(r) + 4*(r^2*e^lamb(r) - r^2)*omega(r, th)^2 - 12*r*e^nu(r)*diff(m0(r), r) - 12*e^nu(r)*m0(r) - 4*(2*Omega1*r^3*omega(r, th) - 2*r^3*omega(r, th)^2 + 3*r*e^nu(r)*h0(r) - 3*r*e^nu(r)*m0(r))*diff(lamb(r), r) - 4*(2*Omega1*r^3*omega(r, th) - r^3*omega(r, th)^2)*diff(nu(r), r))*sin(th)^2 - 54043195528445952*r*e^nu(r)*diff(m0(r), r) - (9007199254740992*r^4*diff(omega(r, th), r)^2 + 18014398509481984*r^4*omega(r, th)*diff(omega(r, th), r, r) + 9007199254740992*r^2*e^lamb(r)*diff(omega(r, th), th)^2 + 18014398509481984*r^2*e^lamb(r)*omega(r, th)*diff(omega(r, th), th, th) + 6*r^2*e^nu(r)*diff(h0(r), r, r) + 12*(e^lamb(r) - 2251799813685248)*e^nu(r)*m0(r) + 4503599627370496*(7*r^2*e^lamb(r) - 8*r^2)*omega(r, th)^2 - 13510798882111488*r*e^nu(r)*diff(h0(r), r) - 40532396646334464*r*e^nu(r)*diff(m0(r), r) + 27021597764222976*e^nu(r)*h0(r) + 6*e^(lamb(r) + nu(r))*h2(r) + 27021597764222976*e^(lamb(r) + nu(r))*m2(r) + (4503599627370496*r^3*omega(r, th)^2 - 3*r^2*e^nu(r)*diff(h0(r), r) - 40532396646334464*r*e^nu(r)*h0(r) + 40532396646334464*r*e^nu(r)*m0(r))*diff(lamb(r), r) + 3*(4503599627370496*r^3*omega(r, th)^2 + 2*r^2*e^nu(r)*diff(h0(r), r) - r^2*e^nu(r)*diff(m0(r), r) - 4503599627370496*r*e^nu(r)*h0(r) + 4503599627370496*r*e^nu(r)*m0(r))*diff(nu(r), r) - 9007199254740992*(r^4*omega(r, th)*diff(lamb(r), r) + r^4*omega(r, th)*diff(nu(r), r) - 8*r^3*omega(r, th))*diff(omega(r, th), r))*LP2(th) + 6*e^(lamb(r) + nu(r))*h2(r) + 27021597764222976*e^(lamb(r) + nu(r))*k2(r) + 27021597764222976*e^(lamb(r) + nu(r))*m2(r) - (1125899906842624*r^3*omega(r, th)^2 + 3*r^2*e^nu(r)*diff(h0(r), r) + 54043195528445952*r*e^nu(r)*h0(r) - 54043195528445952*r*e^nu(r)*m0(r))*diff(lamb(r), r) - (21392098230009856*r^3*omega(r, th)^2 - 6*r^2*e^nu(r)*diff(h0(r), r) + 3*r^2*e^nu(r)*diff(m0(r), r))*diff(nu(r), r) - 4503599627370496*(2*r^4*omega(r, th)*diff(lamb(r), r) + 2*r^4*omega(r, th)*diff(nu(r), r) - 17*r^3*omega(r, th))*diff(omega(r, th), r))*e^(-lamb(r) - nu(r))/(pi*r^2*sin(th)^2)

In [37]:
ddEq11 = ddG[1,1] - kappa*ddT[1,1] == 0
P02value = solve(ddEq11, P02)[0].rhs()

P02value_paper = ((4/r^2)*(e^(-lamb)*r*diff(h0,r)\
                  -m0*(8*pi*r^2*P+1))\
                  +1/3*r^2*jj^2*diff(omega,r)^2)/(8*pi)

P02value_check = (P02value - P02value_paper)
P02value_check = P02value_check.simplify_full()

In [38]:
ddEq03 = ddG[0,3] - kappa*ddT[0,3] == 0
ddEq03 = ddEq03*(3/(16*pi*r^2*Omega2))
ddEq03 = ddEq03.simplify_full()

ddEq12 = ddG[1,2] - kappa*ddT[1,2] == 0
ddEq12 = ddEq12*(r*e^(nu))
ddEq12 = ddEq12.simplify_full()